# SVMによるボクセルごとの学習と性能評価（単変量解析）
----

引数：raw_rock.csv/raw_scissor.csv/raw_paper.csvがあるディレクトリまでのパス

----

入力：raw_rock.csv/raw_scissor.csv/raw_paper.csv

----

出力：ACCURACY[loo or k_cv]_VOXunivariate_SVM.csv ボクセルごとの識別性能評価結果一覧

----

ボクセルごとに単変量解析を行う．  
k分割交差検証法により1グループをテストデータの，k-1グループを教師データとし，SVMを用いて学習，精度評価． 
ベクトル：各ボクセルにおけるある時刻のZ-score（1ベクトル）

In [12]:
print('############ ML_SVM_VOXunivariate_kCV.py program excution ############')

############ ML_SVM_VOXunivariate_kCV.py program excution ############


In [13]:
import numpy as np
import pandas as pd
import sys
from sklearn import cross_validation
from sklearn import svm
from sklearn.model_selection import train_test_split

In [17]:
#args = sys.argv
#PATH = args[1]

# jupyter notebookのときはここで指定
PATH = '../State-2fe_SpmActive/20181029rn/mb/RawData/'

# 検証手法
kCV = 10

# 検証手法
col_name = str(kCV) + 'CV'

## SVM_kCV関数
引く数としてデータをX，ラベルをyで受け取る．  
交差検証法の一つk分割交差検証法で識別精度評価を行う．  

In [18]:
def SVM_kCV(X, y):
    
    # 線形SVMのインスタンスを生成
    model = svm.SVC(kernel = 'linear', C = 1)
    
    # k分割し，1グループをテストデータ，残りグループを教師データにして評価
    # すべてのグループに対して行う
    # 評価結果（識別率）を格納
    CVscore = cross_validation.cross_val_score(model, X, y, cv = kCV)
    
    # 評価結果（識別率）の平均を求める
    result = CVscore.mean()
    
    # パーセントに直す
    result = round(result * 100, 1)

    print('k = ' + str(kCV) + '：' + str(CVscore))
    
    return result
    

# main関数

In [22]:
if __name__ == '__main__':
    
    # 読み込みたいファイルのパス
    PATH_rock = PATH + 'raw_rock.csv'
    PATH_scissor = PATH + 'raw_scissor.csv'
    PATH_paper = PATH + 'raw_paper.csv'
    
    # csvファイル読み込み
    # headerは設定せず，転置後にset_index()する（header = 0にすると列名が変えられる）
    rock = pd.read_csv(PATH_rock, header = 0, index_col = None).T
    scissor = pd.read_csv(PATH_scissor, header = 0, index_col = None).T
    paper = pd.read_csv(PATH_paper, header = 0, index_col = None).T

In [24]:
    # ボクセルの数
    voxNum = len(rock)

    # 全ボクセルの識別率を格納するデータフレーム
    voxAc = pd.DataFrame(index = range(voxNum), columns = [col_name])
    
    counter = 0
    csvcounter = 0
    voxNames = []

    for voxNo in range(voxNum):

        voxName = 'Voxel' + str(voxNo + 1)
        print(voxName + '( ' + str(counter+1) + ' / ' + str(voxNum) + ' )')

        # ボクセルのデータを取得
        rockVox = rock.loc[voxName]
        scissorVox = scissor.loc[voxName]
        paperVox = paper.loc[voxName]

        # データセット作成
        rockVox_vec = np.ravel(rockVox)
        scissorVox_vec = np.ravel(scissorVox)
        paperVox_vec = np.ravel(paperVox)

        data = np.r_[rockVox_vec, scissorVox_vec, paperVox_vec]

        # データ数+1にするためにリシェイプ
        data = data.reshape(-1, 1)

        # ラベルを作成
        rockVox_label = np.zeros(len(rockVox_vec))
        scissorVox_label = np.ones(len(scissorVox_vec))
        paperVox_label = np.ones(len(paperVox_vec)) * 2

        labels = np.r_[rockVox_label, scissorVox_label, paperVox_label]
        
        # 学習と評価
        result_vox = SVM_kCV(data, labels)
        print(result_vox)
        
        # データフレームに格納
        voxAc.at[voxNo, :] = result_vox
        
        # 途中経過見る用
        # 何ボクセルで一度出力するか
        midNum = 1000

        if (counter % midNum == 0) and (counter != 0):

            PATH_test = PATH + 'ACMID' + str(csvcounter) + '[' + str(kCV) + 'cv]_VOXunivariate' + '_SVM.csv'
            print(PATH_test)
            MidVoxAc = voxAc.iloc[(csvcounter * midNum):((csvcounter + 1) * midNum), :]
            MidVoxAc.index = voxNames[(csvcounter * midNum):((csvcounter + 1) * midNum)]
            MidVoxAc.to_csv(PATH_test, index = True)

            csvcounter = csvcounter + 1
        
        counter = counter + 1
        voxNames = voxNames + [voxName]
        


Voxel1( 1 / 7 )
k = 10：[ 0.31111111  0.34444444  0.46666667  0.18888889  0.32222222  0.34444444
  0.31034483  0.32183908  0.27586207  0.20689655]
30.9
Voxel2( 2 / 7 )
k = 10：[ 0.24444444  0.24444444  0.26666667  0.21111111  0.24444444  0.3
  0.1954023   0.1954023   0.29885057  0.34482759]
25.5
Voxel3( 3 / 7 )
k = 10：[ 0.26666667  0.25555556  0.34444444  0.23333333  0.26666667  0.27777778
  0.20689655  0.22988506  0.22988506  0.27586207]
25.9
Voxel4( 4 / 7 )
k = 10：[ 0.32222222  0.3         0.33333333  0.18888889  0.33333333  0.3
  0.25287356  0.29885057  0.20689655  0.32183908]
28.6
Voxel5( 5 / 7 )
k = 10：[ 0.4         0.26666667  0.36666667  0.28888889  0.32222222  0.28888889
  0.31034483  0.37931034  0.1954023   0.33333333]
31.5
Voxel6( 6 / 7 )
k = 10：[ 0.33333333  0.3         0.41111111  0.33333333  0.26666667  0.34444444
  0.20689655  0.26436782  0.2183908   0.26436782]
29.4
Voxel7( 7 / 7 )
k = 10：[ 0.32222222  0.33333333  0.32222222  0.3         0.32222222  0.32222222
  0.33333333

In [25]:
    # 行名つける
    voxAc.index = voxNames

    # csv書き出し
    PATH_RESULT = PATH + 'ACCURACY[' + str(kCV) + 'CV]_VOXunivariate' + '_SVM.csv'
    voxAc.to_csv(PATH_RESULT, index = True)